In [ ]:
import json
import re
from tqdm import tqdm

In [ ]:
with open("news.json") as f:
    news_list = json.load(f)

In [ ]:
documents = []
invalid_line_patterns = re.compile(
    r"무단\s*전재|배포\s*금지|Copyrights|관련기사|기사\s*제보|여러분의 제보|카카오톡\s*:s*"
)

for article in tqdm(news_list, mininterval=1):
    text = article["text"]

    lines = text.split("\n")
    filtered_lines = []

    for line in lines:
        line = " ".join(line.split())

        # OOO 기자 패턴 제거
        line = re.sub(r"\w+ 기자", "", line)
        # email 주소 제거
        line = re.sub(r"\w+@\w+\.\w+", "", line)
        # 뉴스에서 []는 [논산] [대구=뉴시스] 등 다양한 참조를 의미하므로 제거
        line = re.sub(r"\[.*\]", "", line)
        # (서울=연합뉴스) 같은 패턴 제거
        line = re.sub(r"\(.*=.*\)", "", line)
        # 2024.10.29/뉴스1 같은 패턴 제거
        line = re.sub(r"\d{4}\.\d{2}\.\d{2}/.*\b", "", line)

        # 공백 제거
        line = " ".join(line.split())

        if invalid_line_patterns.search(line):
            # 무단전재, 배포금지 등이 포함된 문장 이후 문장들은 제외
            break

        # 한국어가 50자 이상 포함된 경우만 포함
        num_korean_chars = len(re.findall(r"[ㄱ-ㅎ가-힣]", line))
        if num_korean_chars >= 10:
            filtered_lines.append(line)

    text = "\n".join(filtered_lines)
    # 한국어가 10자 이상 포함된 경우만 포함
    num_korean_chars = len(re.findall(r"[ㄱ-ㅎ가-힣]", text))
    if num_korean_chars >= 50:
        documents.append(text)

# 중복 제거
documents = list(set(documents))

In [ ]:
# 추후 시각화 관련 코드
from collections import defaultdict
from bokeh.plotting import figure, show, output_file
from bokeh.palettes import Category10
from bokeh.models import HoverTool, ColumnDataSource

output_file("t-sne.html")

topic_idx_per_document = W.argmax(axis=1)
document_idx_per_topic = defaultdict(list)

source_per_topic = {}
for doc_idx, topic_idx in enumerate(topic_idx_per_document):
    document_idx_per_topic[topic_idx].append(doc_idx)
    source_per_topic[topic_idx] = ColumnDataSource(
        data=dict(
            x=W2[document_idx_per_topic[topic_idx], 0],
            y=W2[document_idx_per_topic[topic_idx], 1],
            text=[
                document_tokens_list[doc_idx][0][:100]
                for doc_idx in document_idx_per_topic[topic_idx]
            ],
        )
    )
    
p = figure(title="t-SNE scatter plot")
p.width = 960
p.height = 720
p.add_tools(HoverTool(tooltips=[("기사", "@text")], attachment="right"))
for topic_idx, source in source_per_topic.items():
    p.scatter(
        x="x",
        y="y",
        source=source,
        size=5,
        fill_alpha=0.5,
        legend_label=f"Topic #{topic_idx}",
        color=Category10[10][topic_idx],
    )

p.legend.location = "top_right"
show(p)